In [9]:
import numpy as np
from setup_mnist import *
from keras.models import Input, Model
from keras.layers import Average
import tensorflow as tf
from noise_functions_dl import GradientDescentDL, gradientDescentFunc
from functools import partial
from mwu import adversary
import time
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data = MNIST()
X_exp = np.load("multiclass_data_2/X_exp.npy")
Y_exp = np.load("multiclass_data_2/Y_exp.npy")
Target_exp = np.load("multiclass_data_2/Target_exp.npy")

In [3]:
X_exp = X_exp.reshape(-1, 28, 28, 1)
Y_exp = np.array([(np.arange(10) == l).astype(np.float32) for l in Y_exp])
Target_exp = np.array([(np.arange(10) == l).astype(np.float32) for l in Target_exp])

In [4]:
def ensembleModels(models, model_input):
    # taken from https://medium.com/@twt446/ensemble-and-store-models-in-keras-2-x-b881a6d7693f
    # collect outputs of models in a list
    yModels=[model(model_input) for model in models] 
    # averaging outputs
    yAvg=Average()(yModels) 
    # build model from same input and avg output
    modelEns = Model(inputs=model_input, outputs=yAvg, name='ensemble')  
    return modelEns

In [5]:
model_dir = "deep_networks"
models = [conv_net(False, 2, 200, model_dir + "/conv0"), conv_net(True, 2, 200, model_dir + "/conv1"), 
          conv_net(True, 4, 64, model_dir + "/conv2"), multilayer(4, 128, model_dir + "/mlp0"),
          multilayer(2, 256, model_dir + "/mlp1")]

In [6]:
[model.evaluate(data.test_data, data.test_labels) for model in models]

10000/10000 [==============================] - 1s 68us/step


[[0.0576457683640132, 0.9874],
 [0.03487977762353567, 0.992],
 [0.034175702735514325, 0.9932],
 [0.1319694802839309, 0.9614],
 [0.07794954743353882, 0.9757]]

In [7]:
[model.evaluate(X_exp, Y_exp) for model in models]

100/100 [==============================] - 0s 107us/step


[[0.0007815231964968916, 1.0],
 [0.0009589233167935163, 1.0],
 [9.794235913432203e-05, 1.0],
 [0.00961893867701292, 1.0],
 [0.008963608406484127, 1.0]]

In [53]:
s = time.time()
alpha = 20
lr = .1
opt_iters=3000

with tf.Session() as sess:
    
    model_dir = "deep_networks"
    models = [conv_net(False, 2, 200, model_dir + "/conv0"), conv_net(True, 2, 200, model_dir + "/conv1"), 
              conv_net(True, 4, 64, model_dir + "/conv2"), multilayer(4, 128, model_dir + "/mlp0"),
              multilayer(2, 256, model_dir + "/mlp1")]
    model_input = Input(shape=models[0].input_shape[1:])
    ensemble = ensembleModels(models, model_input)
    ensemble.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])
#     print [model.evaluate(X_exp, Y_exp) for model in models]
#     print ensemble.evaluate(X_exp, Y_exp)
    attack_obj = GradientDescentDL(sess, [ensemble], alpha, (28, 1, 10), (-.5, .5), 
                                   targeted=False, batch_size=1, max_iterations=opt_iters,
                                   learning_rate=lr, confidence=0)
    noise_func = partial(gradientDescentFunc, attack=attack_obj)
    
    ens = adversary(np.array([1.0]), [ensemble], X_exp, Y_exp, alpha, noise_func, False)
print time.time() - s


4570.92199111


In [ ]:
np.save("ensemble_sol_mnist_dl_untargeted_2_8.npy", V)

In [57]:
model_dir = "deep_networks"
models = [conv_net(False, 2, 200, model_dir + "/conv0"), conv_net(True, 2, 200, model_dir + "/conv1"), 
          conv_net(True, 4, 64, model_dir + "/conv2"), multilayer(4, 128, model_dir + "/mlp0"),
          multilayer(2, 256, model_dir + "/mlp1")]

In [ ]:
ensemble_noise_ut = np.load("ensemble_sol_mnist_dl_untargeted_2_8.npy")

In [58]:
ensemble_res_ut_full = [model.evaluate(X_exp + ens, Y_exp)[1] for model in models]

100/100 [==============================] - 1s 9ms/step


In [61]:
np.save("yaron_ensemble_comparison.npy", ens)

In [63]:
noised_ims = X_exp + ens

In [68]:
from scipy.misc import imsave, imresize

In [69]:
for i, im in enumerate(noised_ims):
    imsave("yaron_pictures/ensemble/{}.png".format(i), imresize(im.reshape(28,28), (300, 300)))

/Users/juanperdomo/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  from ipykernel import kernelapp as app
/Users/juanperdomo/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  from ipykernel import kernelapp as app


In [64]:
noised_ims.shape

(100, 28, 28, 1)

In [54]:
t = [np.linalg.norm(x) for x in ens]

In [55]:
np.mean(t)

5.140512

In [59]:
ensemble_res_ut_full

[0.48, 0.43, 0.5, 0.03, 0.04]

In [ ]:
noised_ims = X_exp + ens

In [24]:
ensemble_res_ut = max(ensemble_res_ut_full)

In [25]:
ensemble_res_ut

0.75

Ensemble Baseline is .87

In [ ]:
alpha = 2.8
lr = .001
opt_iters=3000
with tf.Session() as sess:
    
    model_dir = "deep_networks"
    models = [conv_net(False, 2, 200, model_dir + "/conv0"), conv_net(True, 2, 200, model_dir + "/conv1"), 
              conv_net(True, 4, 64, model_dir + "/conv2"), multilayer(4, 128, model_dir + "/mlp0"),
              multilayer(2, 256, model_dir + "/mlp1")]
    model_input = Input(shape=models[0].input_shape[1:])
    individual_noise = []
    for model in models:
        attack_obj = GradientDescentDL(sess, [model], alpha, (28, 1, 10), (-.5, .5), 
                                       targeted=False, batch_size=1, max_iterations=opt_iters,
                                       learning_rate=lr, confidence=0)
        noise_func = partial(gradientDescentFunc, attack=attack_obj)
        V = adversary(np.array([1.0]), [model], X_exp, Y_exp, alpha, noise_func, False)
        individual_noise.append(V)

In [ ]:
np.save("mnist_dl_individual_baselines.npy", np.array(individual_noise))

In [ ]:
individual_noise_ut = np.load("mnist_dl_individual_baselines.npy")

In [ ]:
best_individual_ut_full = ([([model.evaluate(X_exp + noise, Y_exp)[1] for model in models]) 
                           for noise in individual_noise_ut])

In [ ]:
best_individual_ut_full

In [ ]:
best_individual_ut = min([max(individual_res) for individual_res in best_individual_ut_full])

In [ ]:
best_individual_ut

Best Individual Model is .99

# Targeted

In [ ]:
alpha = 3.0
lr = .001
opt_iters=3000

with tf.Session() as sess:
    model_dir = "deep_networks"
    models = [conv_net(False, 2, 200, model_dir + "/conv0"), conv_net(True, 2, 200, model_dir + "/conv1"), 
              conv_net(True, 4, 64, model_dir + "/conv2"), multilayer(4, 128, model_dir + "/mlp0"),
              multilayer(2, 256, model_dir + "/mlp1")]
    model_input = Input(shape=models[0].input_shape[1:])
    ensemble = ensembleModels(models, model_input)
    ensemble.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])
    print [model.evaluate(X_exp, Target_exp) for model in models]
    print ensemble.evaluate(X_exp, Target_exp)
    attack_obj = GradientDescentDL(sess, [ensemble], alpha, (28, 1, 10), (-.5, .5), 
                                   targeted=True, batch_size=1, max_iterations=opt_iters,
                                   learning_rate=lr, confidence=0)
    
    noise_func = partial(gradientDescentFunc, attack=attack_obj)
    targeted_ensemble = adversary(np.array([1.0]), [ensemble], X_exp, Y_exp, alpha, noise_func, Target_exp)

In [ ]:
np.save("ensemble_sol_mnist_dl_targeted_3_0.npy", np.array(targeted_ensemble))

In [ ]:
ensemble_noise_t = np.load("ensemble_sol_mnist_dl_targeted_3_0.npy")

In [ ]:
ensemble_res_t_full = [model.evaluate(X_exp + ensemble_noise_t, Target_exp)[1] for model in models]

In [ ]:
ensemble_res_t_full

In [ ]:
ensemble_res_t = min(ensemble_res_t_full)

In [ ]:
ensemble_res_t

Targeted Ensemble Baseline is 0.04

In [ ]:
alpha = 3.0
lr = .001
opt_iters=3000
with tf.Session() as sess:
    model_dir = "deep_networks"
    models = [conv_net(False, 2, 200, model_dir + "/conv0"), conv_net(True, 2, 200, model_dir + "/conv1"), 
              conv_net(True, 4, 64, model_dir + "/conv2"), multilayer(4, 128, model_dir + "/mlp0"),
              multilayer(2, 256, model_dir + "/mlp1")]
    individual_noise_targeted = []
    for model in models:
        attack_obj = GradientDescentDL(sess, [model], alpha, (28, 1, 10), (-.5, .5), 
                                       targeted=True, batch_size=1, max_iterations=opt_iters,
                                       learning_rate=lr, confidence=0)
        noise_func = partial(gradientDescentFunc, attack=attack_obj)
        noise = adversary(np.array([1.0]), [model], X_exp, Y_exp, alpha, noise_func, Target_exp)
        individual_noise_targeted.append(noise)

In [ ]:
np.save("mnist_dl_individual_baselines_targeted.npy", np.array(individual_noise_targeted))

In [ ]:
individual_noise_t = np.load("mnist_dl_individual_baselines_targeted.npy")

In [ ]:
individual_noise_t_full = ([([model.evaluate(X_exp + noise, Target_exp)[1] for model in models]) 
                             for noise in individual_noise_t])

In [ ]:
individual_noise_t_full

In [ ]:
best_individual_t = max([min(res) for res in individual_noise_t_full])

In [ ]:
best_individual_t

Max_Model baseline is 0 for MNIST

In [ ]:
min_acc_t = np.load("deepLearning-mnist-targeted-3.0-2-4-2033/acc_history.npy")

In [ ]:
noise_mwu_t = np.load("deepLearning-mnist-targeted-3.0-2-4-2033/noise.npy")

In [ ]:
mwu_t_results = [[model.evaluate(X_exp + noise, Target_exp)[1] for model in models] for noise in noise_mwu_t]

In [ ]:
mwu_t_results = np.array(mwu_t_results)

In [ ]:
np.mean(mwu_t_results, axis=0)

In [ ]:
num_iters = 50
plt.plot(range(num_iters), min_acc_t, color="orange")
plt.plot(range(num_iters), [ensemble_res_t] * num_iters, linestyle="--", color="purple")
# plt.plot(range(num_iters), [best_individual_t] * num_iters, linestyle="--", color="black")
plt.legend(["untargeted loss", "ensemble baseline", "individual baseline"], loc="right", 
           bbox_to_anchor=(.95,.4), fontsize=12)
plt.title("Universal Noise Deep Learning: MNIST")
plt.ylabel("Max Accuracy")
plt.xlabel("MWU Round")

# NEed to move these cells up

In [ ]:
max_acc_ut = np.load("deepLearning-mnist-untargeted-2.8-2-4-1951/acc_history.npy")

In [ ]:
num_iters = 50
ensemble_res = .99
plt.plot(range(num_iters), max_acc_ut, color="orange")
plt.plot(range(num_iters), [ensemble_res_ut] * num_iters, linestyle="--", color="purple")
plt.plot(range(num_iters), [best_individual_ut] * num_iters, linestyle="--", color="black")
plt.legend(["untargeted loss", "ensemble baseline", "individual baseline"], loc="right", 
           bbox_to_anchor=(.95,.4), fontsize=12)
plt.title("Universal Noise Deep Learning: MNIST")
plt.ylabel("Max Accuracy")
plt.xlabel("MWU Round")

In [ ]:
mwu_noise = np.load("deepLearning-mnist-untargeted-2.8-2-4-1951/noise.npy")

In [ ]:
mwu_accuracies = [[model.evaluate(X_exp + noise, Y_exp)[1] for model in models] for noise in mwu_noise]

In [ ]:
mwu_accuracies = np.array(mwu_accuracies)

In [ ]:
np.mean(mwu_accuracies, axis=0)